In [1]:
import os
import numpy as np
#os.environ['AUTOGRAPH_VERBOSITY'] = '0'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

In [2]:
print(tf.__version__)

2.5.0


In [3]:
import pickle
with open('/root/autodl-nas/workspace/datasets/kuairec/small_matrix.pkl', 'rb') as f:    
    user_dict = pickle.load(f)
items_num=max(max(user_dict.values()))
maxlen=1000
len_seq=100
batch_size=512
epoch_num=100
hidden_size=64
keep_rate=0.9
layers_num=5
num_interest=1
neg_num=4
test_neg_num=100

In [4]:
import random
import numpy as np
def sample(user_dict,maxlen,len_seq):
    train_set=[]
    train_val_set=[]
    val_set=[]
    test_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        hist=hist[-maxlen:-2]
        #print(hist)
        for i in range(1,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #print(hist[0:i])
            seq[max(0,len_seq+idx-1):]=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            train_set.append((u,list(seq),nxt))
            #print(seq)
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        test_set.append((u,list(seq),nxt))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set,val_set,train_val_set
            
def non_zero_sample(user_dict,maxlen,len_seq):
    train_set=[]
    test_set=[]
    val_set=[]
    train_val_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        seq = np.zeros([len_seq], dtype=np.int32)
        #print(np.shape(seq[-maxlen:]))
        while(len(hist)<maxlen):
            hist.insert(0,0)
        #print(len(hist),hist)
        hist=hist[-maxlen:-2]
        for i in range(len_seq,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #rint(hist[max(0,i-len_seq):i])
            seq=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
            train_set.append((u,list(seq),nxt,neg_item))
            #print(seq)
        #print(np.shape(hist[0:len(hist)-1]))
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
        val_set.append((u,list(seq),nxt,neg_item))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        neg_item = [random.randint(1, items_num) for _ in range(test_neg_num)]
        test_set.append((u,list(seq),nxt,neg_item))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set ,val_set,train_val_set   

In [5]:
'''train_set,test_set,val_set,train_val_set=non_zero_sample(user_dict,maxlen,len_seq)
print(len(train_set),len(test_set),len(val_set),len(train_val_set))
users_num=len(test_set)'''

'train_set,test_set,val_set,train_val_set=non_zero_sample(user_dict,maxlen,len_seq)\nprint(len(train_set),len(test_set),len(val_set),len(train_val_set))\nusers_num=len(test_set)'

In [6]:
'''users, click_seqs, pos_items,neg_items = zip(*train_set)
train_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}
users, click_seqs, pos_items,neg_items = zip(*test_set)
test_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}'''

"users, click_seqs, pos_items,neg_items = zip(*train_set)\ntrain_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}\nusers, click_seqs, pos_items,neg_items = zip(*test_set)\ntest_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}"

In [7]:
with open('/root/autodl-nas/workspace/datasets/kuairec/small_matrix_train_data.pkl', 'rb') as f:
    train_data=pickle.load(f)
with open('/root/autodl-nas/workspace/datasets/kuairec/small_matrix_test_data.pkl', 'rb') as f:
    test_data=pickle.load(f)
with open('/root/autodl-nas/workspace/datasets/kuairec/small_matrix_val_data.pkl', 'rb') as f:
    val_data=pickle.load(f)

In [8]:
print(np.shape(train_data['neg_item']),np.shape(test_data['neg_item']),np.shape(val_data['neg_item']))

(1267078, 4) (1411, 100) (1411, 4)


In [9]:
'''for i in range(len(train_data['click_seq'])):
    if i%500000==0:
        print(i)
    if list(test_data['click_seq'][0])==list(train_data['click_seq'][i]):
        print(test_data['click_seq'][0],test_data['pos_item'][0],train_data['click_seq'][i],train_data['pos_item'][i])
for i in range(len(train_data['click_seq'])):
    if i%500000==0:
        print(i)
    if list(val_data['click_seq'][0])==list(train_data['click_seq'][i]):
        print(val_data['click_seq'][0],val_data['pos_item'][0],train_data['click_seq'][i],train_data['pos_item'][i])
print('pass')'''

"for i in range(len(train_data['click_seq'])):\n    if i%500000==0:\n        print(i)\n    if list(test_data['click_seq'][0])==list(train_data['click_seq'][i]):\n        print(test_data['click_seq'][0],test_data['pos_item'][0],train_data['click_seq'][i],train_data['pos_item'][i])\nfor i in range(len(train_data['click_seq'])):\n    if i%500000==0:\n        print(i)\n    if list(val_data['click_seq'][0])==list(train_data['click_seq'][i]):\n        print(val_data['click_seq'][0],val_data['pos_item'][0],train_data['click_seq'][i],train_data['pos_item'][i])\nprint('pass')"

In [10]:
def scaled_dot_product_attention(q, k, v, mask):
    """Attention Mechanism Function.
    Args:
        :param q: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param k: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param v: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param mask: A 3d/4d tensor with shape of (None, ..., seq_len, 1)
    :return:
    """
    mat_qk = tf.matmul(q, k, transpose_b=True)  # (None, seq_len, seq_len)
    # Scaled
    dk = tf.cast(k.shape[-1], dtype=tf.float32)
    scaled_att_logits = mat_qk / tf.sqrt(dk)

    paddings = tf.ones_like(scaled_att_logits) * (-2 ** 32 + 1)  # (None, seq_len, seq_len)
    if mask!=None:
        outputs = tf.where(tf.equal(mask, tf.zeros_like(mask)), paddings, scaled_att_logits)  # (None, seq_len, seq_len)
    else:
        outputs=scaled_att_logits
    # softmax
    outputs = tf.nn.softmax(logits=outputs)  # (None, seq_len, seq_len)
    outputs = tf.matmul(outputs, v)  # (None, seq_len, dim)

    return outputs


def split_heads(x, seq_len, num_heads, depth):
    """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    Args:
        :param x: A Tensor with shape of [batch_size, seq_len, num_heads * depth]
        :param seq_len: A scalar(int).
        :param num_heads: A scalar(int).
        :param depth: A scalar(int).
    :return: A tensor with shape of [batch_size, num_heads, seq_len, depth]
    """
    x = tf.reshape(x, (-1, seq_len, num_heads, depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

In [11]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        """Multi Head Attention Mechanism.
        Args:
            :param d_model: A scalar. The self-attention hidden size.
            :param num_heads: A scalar. Number of heads. If num_heads == 1, the layer is a single self-attention layer.
        :return:
        """
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads

        self.wq = tf.keras.layers.Dense(d_model, activation=None)
        self.wk = tf.keras.layers.Dense(d_model, activation=None)
        self.wv = tf.keras.layers.Dense(d_model, activation=None)

    def call(self, q, k, v, mask):
        q = self.wq(q)  # (None, seq_len, d_model)
        k = self.wk(k)  # (None, seq_len, d_model)
        v = self.wv(v)  # (None, seq_len, d_model)
        # split d_model into num_heads * depth
        seq_len, d_model = q.shape[1], q.shape[2]
        q = split_heads(q, seq_len, self.num_heads, q.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        k = split_heads(k, seq_len, self.num_heads, k.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        v = split_heads(v, seq_len, self.num_heads, v.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        # mask
        if mask!=None:
            mask = tf.tile(tf.expand_dims(mask, axis=1), [1, self.num_heads, 1, 1])  # (None, num_heads, seq_len, 1)
        # attention
        scaled_attention = scaled_dot_product_attention(q, k, v, mask)  # (None, num_heads, seq_len, d_model // num_heads)
        # reshape
        outputs = tf.reshape(tf.transpose(scaled_attention, [0, 2, 1, 3]), [-1, seq_len, d_model])  # (None, seq_len, d_model)
        return outputs


class FFN(tf.keras.layers.Layer):
    def __init__(self, hidden_unit, d_model):
        """Feed Forward Network.
        Args:
            :param hidden_unit: A scalar.
            :param d_model: A scalar.
        :return:
        """
        super(FFN, self).__init__()
        self.conv1 = tf.keras.layers.Conv1D(filters=hidden_unit, kernel_size=1, activation='relu', use_bias=True)
        self.conv2 = tf.keras.layers.Conv1D(filters=d_model, kernel_size=1, activation=None, use_bias=True)

    def call(self, inputs):
        x = self.conv1(inputs)
        output = self.conv2(x)
        return output


class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads=1, ffn_hidden_unit=128, dropout=0., layer_norm_eps=1e-6):
        """Encoder Layer.
        Args:
            :param d_model: A scalar. The self-attention hidden size.
            :param num_heads: A scalar. Number of heads.
            :param ffn_hidden_unit: A scalar. Number of hidden unit in FFN
            :param dropout: A scalar. Number of dropout.
            :param layer_norm_eps: A scalar. Small float added to variance to avoid dividing by zero.
        :return:
        """
        super(TransformerEncoder, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = FFN(ffn_hidden_unit, d_model)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)

        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)

    def call(self, inputs):
        x, mask= inputs
        # self-attention
        att_out = self.mha(x, x, x, mask)  # (None, seq_len, d_model)
        att_out = self.dropout1(att_out)
        # residual add
        out1 = self.layernorm1(x + att_out)  # (None, seq_len, d_model)
        # ffn
        ffn_out = self.ffn(out1)
        ffn_out = self.dropout2(ffn_out)
        # residual add
        out2 = self.layernorm2(out1 + ffn_out)  # (None, seq_len, d_model)
        return out2

class Bilinear(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Bilinear, self).__init__()
        self.linear_act = tf.keras.layers.Dense(units, activation=None, use_bias=True)
        self.linear_noact = tf.keras.layers.Dense(units, activation=None, use_bias=False)

    def call(self, a, b, gate_b=None):
        if gate_b is None:
            return tf.keras.activations.sigmoid(self.linear_act(a) + self.linear_noact(b))
        else:
            return tf.keras.activations.tanh(self.linear_act(a) + tf.math.multiply(gate_b, self.linear_noact(b)))
    
'''class AUGRU(tf.keras.layers.Layer):
    def __init__(self, units):
        super(AUGRU, self).__init__()

        self.u_gate = Bilinear(units)
        self.r_gate = Bilinear(units)
        self.c_memo = Bilinear(units)

    def call(self, inputs, state, att_score):
        u = self.u_gate(inputs, state)
        r = self.r_gate(inputs, state)
        c = self.c_memo(inputs, state, r)

        u_= att_score * u
        final = (1 - u_) * state + u_ * c

        return final'''

class AUGRU(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        self.units = units
        self.state_size = units
        super(AUGRU, self).__init__(**kwargs)
        self.u_gate = Bilinear(units)
        self.r_gate = Bilinear(units)
        self.c_memo = Bilinear(units)
    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                        initializer='uniform',
                                        name='kernel')
        self.recurrent_kernel = self.add_weight(
              shape=(self.units, self.units),
              initializer='uniform',
              name='recurrent_kernel')
        self.built = True
    def call(self, inputs, state,att_score):
        u = self.u_gate(inputs, state)
        r = self.r_gate(inputs, state)
        c = self.c_memo(inputs, state, r)
        
        u_= att_score * u
        final = (1 - u_) * state + u_ * c
        return final


In [12]:
class dien(tf.keras.models.Model):
    def __init__(self):
        super(dien, self).__init__()
        blocks=1
        embed_reg=0.
        layer_norm_eps=1e-6
        num_heads=1
        use_l2norm=False
        
        self.len_seq=len_seq
        self.item_embedding = tf.keras.layers.Embedding(items_num+1,hidden_size,input_length=self.len_seq,
                                                       embeddings_initializer='random_normal',
                                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.dense_0 = tf.keras.layers.Dense(32, activation=None, use_bias=False)
        self.dense_1 = tf.keras.layers.Dense(1, activation=None, use_bias=True)
        self.hist_gru = tf.keras.layers.GRU(hidden_size, return_sequences=True)
        self.hist_augru = AUGRU(hidden_size)

    def call(self, inputs):
        seq_embed=self.item_embedding(inputs['click_seq'])
        pos_emb=self.item_embedding(tf.reshape(inputs['pos_item'], [-1, ]))
        neg_emb=self.item_embedding(inputs['neg_item'])
        
        #item_join_emb, item_bias, hist_join_emb = self.get_emb(user, item, history)

        seq_gru_embed = self.hist_gru(seq_embed)
        
        #hist_mask = tf.sequence_mask(length, max(length), dtype=tf.bool)
        #hist_mask = tf.tile(tf.expand_dims(hist_mask, -1), (1,1,self.item_dim+self.cate_dim))
        
        hist_attn = tf.nn.softmax(tf.matmul(tf.expand_dims(pos_emb, 1), seq_gru_embed, transpose_b=True))

        seq_hid_embed = tf.zeros_like(seq_gru_embed[:,0,:])
        
        '''for in_emb, in_att in zip(tf.transpose(seq_gru_embed, [1,0,2]),
                                  tf.transpose(hist_attn, [2,0,1])):'''
        
        seq_hid_embed = self.hist_augru(tf.transpose(seq_gru_embed, [1,0,2]), seq_hid_embed, tf.transpose(hist_attn, [2,0,1]))
        
        final_hid=tf.reshape(seq_hid_embed[-1],[-1,hidden_size])
        
        pos_join_emb = tf.concat([pos_emb, final_hid], -1)
        pos_output=self.dense_0(pos_join_emb)
        pos_output=self.dense_1(pos_output)#[b,1]
        
        neg_join_emb = tf.concat([neg_emb, tf.tile(tf.expand_dims(final_hid,1),[1,tf.shape(neg_emb)[1],1])], -1)
        neg_output=self.dense_0(neg_join_emb)
        neg_output=self.dense_1(neg_output)#[b,n,1]
                                  
        output=tf.squeeze(tf.concat([tf.expand_dims(pos_output,1),neg_output],1),2)
        
        logits = tf.keras.activations.sigmoid(output)
        
        loss = tf.reduce_mean(- tf.math.log(logits[:,0]))
        
        self.add_loss(loss)
        
        return logits
    
    def summary(self):
        inputs = {
            'click_seq': tf.keras.layers.Input(shape=(self.len_seq,), dtype=tf.int32),
            'pos_item': tf.keras.layers.Input(shape=(), dtype=tf.int32),
            'neg_item': tf.keras.layers.Input(shape=(1,), dtype=tf.int32)  # suppose neg_num=1
        }
        tf.keras.models.Model(inputs=inputs, outputs=self.call(inputs)).summary()

In [13]:
def hr(rank, k):
    """Hit Rate.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: hit rate.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1
    return res / len(rank)


def mrr(rank, k):
    """Mean Reciprocal Rank.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: mrr.
    """
    mrr = 0.0
    for r in rank:
        if r < k:
            mrr += 1 / (r + 1)
    return mrr / len(rank)


def ndcg(rank, k):
    """Normalized Discounted Cumulative Gain.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: ndcg.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1 / np.log2(r + 2)
    return res / len(rank)


def eval_rank(pred_y, metric_names, k=10):
    """Evaluate
        Args:
            :param pred_y: A ndarray.
            :param metric_names: A list like ['hr'].
            :param k: A scalar(int).
        :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    rank = pred_y.argsort().argsort()[:, 0]#最大prob对应的index
    res_dict = {}
    for name in metric_names:
        if name == 'hr':
            res = hr(rank, k)
        elif name == 'ndcg':
            res = ndcg(rank, k)
        elif name == 'mrr':
            res = mrr(rank, k)
        else:
            break
        res_dict[name] = res
    return res_dict

def eval_pos_neg(model, test_data, metric_names, k=10, batch_size=None):
    """Evaluate the performance of Top-k recommendation algorithm.
    Note: Test data must contain some negative samples(>= k) and one positive samples.
    Args:
        :param model: A model built-by tensorflow.
        :param test_data: A dict.
        :param metric_names: A list like ['hr'].
        :param k: A scalar(int).
        :param batch_size: A scalar(int).
    :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    pred_y = - model.predict(test_data, batch_size)
    print(np.shape(pred_y))
    return eval_rank(pred_y, metric_names, k)

In [14]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [15]:
from time import time
strategy = tf.distribute.MirroredStrategy()
batch_size=batch_size*strategy.num_replicas_in_sync
with strategy.scope():
    model = dien()
    model.summary()
    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    model.compile(optimizer=optimizer)
best_eval_dict={'hr':0.,'mrr':0.,'ndcg':0.}
eval_patience={'hr':0.,'mrr':0.,'ndcg':0.}
for epoch in range(1, epoch_num + 1):
    t1 = time()
    model.fit(
            x=train_data,
            epochs=1,
            batch_size=batch_size,
        validation_data=val_data)
    t2 = time()
    eval_dict = eval_pos_neg(model, val_data, ['hr', 'mrr', 'ndcg'], 1, batch_size)
    print('Iteration %d Fit [%.1f s], Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (epoch, t2 - t1, time() - t2, eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))
    for i in eval_dict.keys():
        if eval_dict[i]>=best_eval_dict[i]:
            best_eval_dict[i]=eval_dict[i]
        else:
            eval_patience[i]+=1
    if (np.array(list(eval_patience.values()))>=3).any():
        print('Trigger early stopping')
        break
eval_dict = eval_pos_neg(model, test_data, ['hr', 'mrr', 'ndcg'], 10, batch_size)
print('Iteration %d Fit [%.1f s], Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (epoch, t2 - t1, time() - t2, eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))        

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
tf.reshape (TFOpLambda)         (None,)              0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple    

In [16]:
'''def train_model(model,ds_train,ds_valid,epoches):

    for epoch in tf.range(1,epoches+1):
        model.reset_metrics()
        
        # 在后期降低学习率
        if epoch == 5000:
            model.optimizer.lr.assign(model.optimizer.lr/2.0)
            tf.print("Lowering optimizer Learning Rate...\n\n")
        
        for x, y in ds_train:
            train_result = model.train_on_batch(x, y)

        for x, y in ds_valid:
            valid_result = model.test_on_batch(x, y,reset_metrics=False)
            
        if epoch%100 ==0:
            tf.print("epoch = ",epoch)
            print("train:",dict(zip(model.metrics_names,train_result)))
            print("valid:",dict(zip(model.metrics_names,valid_result)))
            print("")'''

'def train_model(model,ds_train,ds_valid,epoches):\n\n    for epoch in tf.range(1,epoches+1):\n        model.reset_metrics()\n        \n        # 在后期降低学习率\n        if epoch == 5000:\n            model.optimizer.lr.assign(model.optimizer.lr/2.0)\n            tf.print("Lowering optimizer Learning Rate...\n\n")\n        \n        for x, y in ds_train:\n            train_result = model.train_on_batch(x, y)\n\n        for x, y in ds_valid:\n            valid_result = model.test_on_batch(x, y,reset_metrics=False)\n            \n        if epoch%100 ==0:\n            tf.print("epoch = ",epoch)\n            print("train:",dict(zip(model.metrics_names,train_result)))\n            print("valid:",dict(zip(model.metrics_names,valid_result)))\n            print("")'

In [17]:
'''train_model(model,ds_train,ds_test,800)'''

'train_model(model,ds_train,ds_test,800)'

In [18]:
'''
items_num=3416
maxlen=100
len_seq=50
batch_size=512
epoch_num=20
hidden_size=64
keep_rate=0.9
layers_num=2
num_interest=1
neg_num=4
test_neg_num=100

567/567 [==============================] - 15s 15ms/step - loss: 0.3692
Iteration 1 Fit [15.6 s], Evaluate [0.6 s]: HR = 0.1430, MRR = 0.0510, NDCG = 0.0725
567/567 [==============================] - 11s 19ms/step - loss: 0.1491
Iteration 2 Fit [11.0 s], Evaluate [0.3 s]: HR = 0.1912, MRR = 0.0645, NDCG = 0.0942
567/567 [==============================] - 10s 17ms/step - loss: 0.1096
Iteration 3 Fit [10.0 s], Evaluate [0.3 s]: HR = 0.2171, MRR = 0.0739, NDCG = 0.1074
567/567 [==============================] - 10s 18ms/step - loss: 0.0948
Iteration 4 Fit [10.4 s], Evaluate [0.3 s]: HR = 0.2354, MRR = 0.0829, NDCG = 0.1186
567/567 [==============================] - 11s 19ms/step - loss: 0.0859
Iteration 5 Fit [11.1 s], Evaluate [0.3 s]: HR = 0.2470, MRR = 0.0856, NDCG = 0.1235
567/567 [==============================] - 12s 20ms/step - loss: 0.0793
Iteration 6 Fit [11.8 s], Evaluate [0.3 s]: HR = 0.2502, MRR = 0.0906, NDCG = 0.1281
567/567 [==============================] - 12s 21ms/step - loss: 0.0741
Iteration 7 Fit [12.0 s], Evaluate [0.3 s]: HR = 0.2603, MRR = 0.0971, NDCG = 0.1354
567/567 [==============================] - 10s 18ms/step - loss: 0.0702
Iteration 8 Fit [10.3 s], Evaluate [0.2 s]: HR = 0.2709, MRR = 0.1010, NDCG = 0.1410
567/567 [==============================] - 8s 14ms/step - loss: 0.0665
Iteration 9 Fit [7.9 s], Evaluate [0.3 s]: HR = 0.2667, MRR = 0.1004, NDCG = 0.1395
567/567 [==============================] - 9s 16ms/step - loss: 0.0638
Iteration 10 Fit [9.4 s], Evaluate [0.3 s]: HR = 0.2614, MRR = 0.1013, NDCG = 0.1390
567/567 [==============================] - 8s 13ms/step - loss: 0.0613
Iteration 11 Fit [7.9 s], Evaluate [0.3 s]: HR = 0.2637, MRR = 0.1010, NDCG = 0.1394
567/567 [==============================] - 8s 14ms/step - loss: 0.0592
Iteration 12 Fit [8.1 s], Evaluate [0.3 s]: HR = 0.2601, MRR = 0.0992, NDCG = 0.1372
567/567 [==============================] - 8s 13ms/step - loss: 0.0571
Iteration 13 Fit [7.8 s], Evaluate [0.2 s]: HR = 0.2692, MRR = 0.1030, NDCG = 0.1422
567/567 [==============================] - 8s 14ms/step - loss: 0.0556
Iteration 14 Fit [7.9 s], Evaluate [0.3 s]: HR = 0.2594, MRR = 0.1003, NDCG = 0.1378
567/567 [==============================] - 8s 13ms/step - loss: 0.0541
Iteration 15 Fit [7.7 s], Evaluate [0.2 s]: HR = 0.2669, MRR = 0.1037, NDCG = 0.1422
567/567 [==============================] - 8s 13ms/step - loss: nan
Iteration 16 Fit [7.8 s], Evaluate [0.2 s]: HR = 1.0000, MRR = 1.0000, NDCG = 1.0000
567/567 [==============================] - 11s 19ms/step - loss: nan
Iteration 17 Fit [11.2 s], Evaluate [0.2 s]: HR = 1.0000, MRR = 1.0000, NDCG = 1.0000
567/567 [==============================] - 11s 20ms/step - loss: nan
Iteration 18 Fit [11.8 s], Evaluate [0.2 s]: HR = 1.0000, MRR = 1.0000, NDCG = 1.0000
567/567 [==============================] - 10s 17ms/step - loss: nan
Iteration 19 Fit [9.9 s], Evaluate [0.3 s]: HR = 1.0000, MRR = 1.0000, NDCG = 1.0000
567/567 [==============================] - 11s 19ms/step - loss: nan
Iteration 20 Fit [11.2 s], Evaluate [0.3 s]: HR = 1.0000, MRR = 1.0000, NDCG = 1.0000
'''

'\nitems_num=3416\nmaxlen=100\nlen_seq=50\nbatch_size=512\nepoch_num=20\nhidden_size=64\nkeep_rate=0.9\nlayers_num=2\nnum_interest=1\nneg_num=4\ntest_neg_num=100\n\n567/567 [==============================] - 15s 15ms/step - loss: 0.3692\nIteration 1 Fit [15.6 s], Evaluate [0.6 s]: HR = 0.1430, MRR = 0.0510, NDCG = 0.0725\n567/567 [==============================] - 11s 19ms/step - loss: 0.1491\nIteration 2 Fit [11.0 s], Evaluate [0.3 s]: HR = 0.1912, MRR = 0.0645, NDCG = 0.0942\n567/567 [==============================] - 10s 17ms/step - loss: 0.1096\nIteration 3 Fit [10.0 s], Evaluate [0.3 s]: HR = 0.2171, MRR = 0.0739, NDCG = 0.1074\n567/567 [==============================] - 10s 18ms/step - loss: 0.0948\nIteration 4 Fit [10.4 s], Evaluate [0.3 s]: HR = 0.2354, MRR = 0.0829, NDCG = 0.1186\n567/567 [==============================] - 11s 19ms/step - loss: 0.0859\nIteration 5 Fit [11.1 s], Evaluate [0.3 s]: HR = 0.2470, MRR = 0.0856, NDCG = 0.1235\n567/567 [==============================]

In [19]:
!nvidia-smi

Sat Jan  7 17:22:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.76       Driver Version: 515.76       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:43:00.0 Off |                  N/A |
| 45%   45C    P2    83W / 320W |   9137MiB / 10240MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:47:00.0 Off |                  N/A |
| 46%   

In [20]:
a=[[1,2],[3,4],[5,6]]
a=np.asarray(a)
print(a)

[[1 2]
 [3 4]
 [5 6]]


In [21]:
a[[0,1,2],[0,1,0]]

array([1, 4, 5])